In [ ]:
from valurap2 import gcode
from valurap2 import path_planning
from valurap2 import emulate

from ipywidgets import widgets
from ipywidgets import Layout

import imp
imp.reload(gcode)
imp.reload(path_planning)

pp = path_planning.PathPlanner()
pp.emu_in_loop = True

if 1:
    speed_k = 1.0
    pp.delta_e_err = 50
    pp.delta_ve_err = 20
    pp.max_ea = 10000
    pp.accel_step = 5000
else:
    pp.delta_e_err = 100
    pp.delta_ve_err = 100
    pp.max_ea = 20000
    pp.accel_step = 5000
    speed_k = 10.0

all_deltas = []

text = widgets.Textarea(layout=Layout(width='auto', height='450px'))
display(text)

text_val = ""
total_deltas = {}

if 1:
    #fn = "test18.gcode"
    fn = "box20_void_dual.gcode"
    pp.gen_layers(fn, speed_k=speed_k)
else:
    #sg = gcode.file_reader("helmet.gcode")
    sg = gcode.file_reader("test18.gcode")
    for s in sg:
        if isinstance(s, gcode.do_segment):
            hdr = "############# L{} - L{} ###############".format(s.path[0][4], s.path[-1][4])
            print("\n\n{}\n".format(hdr))
            text.value = hdr + "\n...\n...\n" + text_val

            print("path len:", len(s.path), s.path[0:2])
            try:
                path, slowdowns = pp.make_path(s.path, speed_k=speed_k)
                if path is None:
                    continue
                #print(path)
                #print(slowdowns)
                slowdowns, updated, cc = pp.process_corner_errors(path, slowdowns)
                slowdowns, updated = pp.reverse_pass(path, slowdowns)
                slowdowns, updated = pp.forward_pass(path, slowdowns)
                df, profile = pp.gen_segments_float(path, slowdowns)
                print(pp.max_deltas)
                md_str = []
                td_str = []
                for k,v in pp.max_deltas.items():
                    md_str.append("{}: {:10.3f}".format(k, v))
                    total_deltas[k] = max(total_deltas.get(k, 0), v)
                    td_str.append("{}: {:10.3f}".format(k, total_deltas[k]))
                text.value = (hdr + "\n" + " ".join(md_str) + "\n" + " ".join(td_str) + "\n" + text_val)[:10000]
                text_val = text.value

                all_deltas.append(pp.max_deltas.copy())
            except Exception:
                pp.print_msg_buf()
                raise

In [ ]:
import os.path
import pickle

#base_fn = "align3"
base_fn = "test18"
base_fn = "box20_void_dual"
layer = 0
while True:
    fn = "{}_{:05d}.layer".format(base_fn, layer)
    if not os.path.exists(fn):
        break
    print(fn)
    data = pickle.load(open(fn, "rb"))
    for d in data:
        if d[0] == "segment":
            print(d[0], len(d), len(d[1]), len(d[2]))
            for p in d[2][:5]:
                print("   ", p)
        else:
            print(d[0], len(d))
    layer += 1
    if layer > 20:
        break

In [ ]:
steps = emulate.emulate(profile, apg_states=pp.apg_states, no_tracking=False)
display(steps)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
#steps.plot(x="t", y=["apg5_v"], style="*")
steps.plot(x="apg1_x", y=["apg2_x"], style="*")

In [ ]:
from valurap2 import buf_commands
import pickle

imp.reload(buf_commands)
cb = buf_commands.CommandBuffer(debug=True)
cb.add_segments_head(pp)
cb.add_segments(profile)
cb.add_segments_foot()
#print(list(cb.buffer))
pickle.dump(list(cb.buffer), open("commands.pck", "wb"))

In [ ]:
import pandas
import numpy
import imp
from valurap2 import path_planning

imp.reload(path_planning)
pp = path_planning.PathPlanner()
pp.spms = pp.print_spms
pp.init_apgs()
print(pp.apg_states)

pp.ext_to_code([100], axes=["X1"])

In [ ]:
from valurap2 import rest_client

rc = rest_client.Client()
rc.home()
rc.wait_idle()
rc.moveto(X1=0, X2=0, Y=0)
rc.move(E1=10, E2=10)
rc.wait_idle()
rc.exec_binary(cb.buffer)
rc.wait_idle()


In [ ]:
rc.exec_binary(cb.buffer)